In [80]:
import os
import re
import json
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel
import warnings

In [81]:
def data_import(data_name):
    data_path = os.getenv('HOME')+'/aiffel/project_data/dlthon/'+data_name
    imported_data = pd.read_csv(data_path)
    return imported_data

def cleaning_sentence(sentence):
        sentence = sentence.lower()
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = re.sub(r'\([^)]*\)', '', sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub("[^가-힣a-zA-Z0-9\.\?\!,]+", " ", sentence)
        sentence = re.sub(r'[\n\r]+', ' ', sentence)
        sentence = sentence.strip()
        return sentence

def preprocess_sentence(data_list):
    retrun_list = []
    for sentence_frame in data_list:
         befor_df = {}
         conv_data = []
         class_data = []
         class_name = sentence_frame['class'][0]
         for sentence in sentence_frame['conversation']:
             cleaned_sentence = cleaning_sentence(sentence)
             conv_data.append(cleaned_sentence)
             class_data.append(class_name)
         return_df = pd.DataFrame({'class' : class_data, 'conversation': conv_data})
         retrun_list.append(return_df)
    return retrun_list

def random_deletion(text, prob=0.2):
    words = text.split()
    if len(words) == 1:
        return text
    return ' '.join([word for word in words if random.random() > prob])

def random_swap(text, n=1):
    words = text.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def convert_label(data):
    label_map = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
    
    data['class'] = data['class'].map(label_map)
    return data

def cal_len(train, test, rate):
    con_data = np.concatenate((train, test), axis = 0)
    seg_len = []
    spl_len = []
    for i in con_data:
        single_seg_len = len(i)
        seg_len.append(single_seg_len)
    for i in con_data:
        single_spl_len = len(i.split())
        spl_len.append(single_spl_len)
    spl_len.sort()
    seg_len.sort()
    print('spl len is : ', spl_len[int(len(spl_len)*rate)])
    print('seg len is : ', seg_len[int(len(seg_len)*rate)])

In [82]:
def data_aug(data_list, prob, n):
    len_data = []

    for data in data_list:
        len_data.append(len(data))

    max_len_of = max(len_data)

    return_data = []

    for data_set in data_list:
        if len(data_set) != max_len_of:
            conver_data = []
            class_Data = []
            return_df = {}
            aug_len = max_len_of - len(data_set)
            class_name = data_set['class'][0]
            for i in range(aug_len): class_Data.append(class_name)
            for i in range(aug_len):
                choice_num = random.random()
                random_seq = data_set['conversation'].sample(1).iloc[0]
                if choice_num >= 0.5:
                    output_seq = random_deletion(random_seq, prob)
                    conver_data.append(output_seq)
                else:
                    output_seq = random_swap(random_seq, n)
                    conver_data.append(output_seq)
                    
            retrun_df = pd.DataFrame({'class':class_Data, 'conversation':conver_data})
        else:
            retrun_df = 0

        return_data.append(retrun_df)

    final_list = []

    for auged_data, real_data in zip(return_data, data_list):
        if isinstance(auged_data, pd.DataFrame):
            real_data = pd.concat([real_data, auged_data])
            real_data.reset_index(drop=True, inplace=True)
            final_list.append(real_data)
        else:
            final_list.append(real_data)
    
    return final_list


In [7]:
bert_model = TFAutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
def bert_encode(datas, sent_max_length):
    input_ids = []
    attention_masks = []
    
    for sent in datas:
        encoded = tokenizer.encode_plus(sent,
                                        add_special_tokens = True,
                                        max_length = sent_max_length,
                                        padding='max_length',
                                        truncation = True,
                                        return_attention_mask=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

In [9]:
def create_model(bert_model, max_len):
    input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32)
    
    output = bert_model([input_ids, attention_mask])
    output = output.last_hidden_state
    output = tf.keras.layers.Dense(64, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(3, activation='softmax')(output)
    
    model = tf.keras.Model(inputs = [input_ids, attention_mask], outputs = output)
    model.compile(Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
train_data = data_import('train.csv')
nomal_data = data_import('nomal_data.csv')

In [11]:
train_data = train_data.drop(train_data.columns[0], axis=1)
threat_data = train_data[train_data['class'] == '협박 대화']
extort_data = train_data[train_data['class'] == '갈취 대화']
co_bully_data = train_data[train_data['class'] == '직장 내 괴롭힘 대화']
bully_data = train_data[train_data['class'] == '기타 괴롭힘 대화']

threat_data.reset_index(drop=True, inplace=True)
extort_data.reset_index(drop=True, inplace=True)
co_bully_data.reset_index(drop=True, inplace=True)
bully_data.reset_index(drop=True, inplace=True)

k = []
for i in range(nomal_data.shape[0]): k.append('일반 대화')
nomal_data['class'] = k
nomal_data = nomal_data.rename(columns={'0':'conversation'})
nomal_data = nomal_data[['class', 'conversation']]
nomal_data = nomal_data.sample(3000)
nomal_data.reset_index(drop=True, inplace=True)

In [12]:
data_list = [nomal_data, threat_data, extort_data, co_bully_data, bully_data]

In [13]:
nomal_data

,class,conversation
0,일반 대화,카드 결제되나요?\n그럼요. 카드 결제 가능하세요
1,일반 대화,가게 문 언제까지 하는데요?\n6시 반까지해
2,일반 대화,오뎅 국물은 이걸로 뜨나요?\n
3,일반 대화,이 가방은 얼마예요?\n이 가방은 오만 오천 원이에
4,일반 대화,여기는 몇 시에 문 열어요?\n열한시 반에 열어
...,...,...
2995,일반 대화,리필 심을 다 쓰면?\n리필 심이 따로 나와서 갈아 끼우시면 돼
2996,일반 대화,"식빵 다 나갔나요?\n네, 오늘 식빵이 빨리 빠졌어"
2997,일반 대화,"고기를 검은 비닐봉지에 담아 주실 수 있나요?\n네, 다른 건 필요 없으세요?\n현..."
2998,일반 대화,냉장보관 해야하나요?\n냉동실에 보관하시고 당일에 꺼내 냉장으로 옮기셔야 합니다


In [14]:
proto_data = pd.concat(data_list, axis = 0)

proto_data = np.array(proto_data['conversation'])

In [15]:
changed_data = []

for sentence in proto_data:
    sentence = re.sub(r'\n', ' 옌 ', sentence)
    changed_data.append(sentence)

prot_data = np.array(changed_data)

In [16]:
prot_data

array(['카드 결제되나요? 옌 그럼요. 카드 결제 가능하세요', '가게 문 언제까지 하는데요? 옌 6시 반까지해',
       '오뎅 국물은 이걸로 뜨나요? 옌 ', ...,
       '애들아 공연 연습하자. 옌 이번주에 최종 평가 있어. 빨리 하자. 옌 야 김정미 너는 왜 맨날 이 부분에서 틀려? 옌 어. 미안해 내가 더 연습해야 했는데 옌 너 때문에 동작이 하나도 안 맞잖아. 그리고 표정은 또 왜이렇게 어두워? 옌 미안해 다음부턴 웃으면서 할게 옌 춤에 소질이 없으면 그냥 팀에서 나가. 너 케어할 시간 없으니까 옌 미안해 내가 더 노력할게 옌 제발 민폐 좀 끼치지마 대체 춤에 뭐가 자신있다고 여기 들어온건지. 옌 알았어.',
       '나를 왜 자꾸 때려 옌 때리고 싶으니깐. 옌 내가 뭐 잘못했어? 옌 아니 잘못없지 옌 근데 왜? 옌 너만보면 기분이 나쁘니깐 옌 왜 나쁜데 옌 쳐다보는게 기분나쁘다 옌 그럼 어떡해야해? 옌 그냥 닥치고 맞으면 돼',
       '준하야 넌 대가리가 왜이렇게 크냐? 옌 내 머리가 뭐. 옌 밥먹으면 대가리만 크냐 너는? 옌 아니. 무슨 머리만 큰다고 그러는거야 옌 주위를 둘러봐라. 내가 살면서 너보다 큰 대가리를 못봤어. 옌 나한테 왜그러는거야. 옌 무식하게 대가리만 크면 뭐하냐? 멍청해가지고는 쯧쯧 옌 넌 내가 그렇게 싫니? 옌 당연하지. 언제 이 세상에서 없어지냐 너 내가 그날만 기다리고 있어 옌 조만간 사라져줄게 너 눈앞에서 옌 그거 요즘들은 소식중에 제일 좋은 소식이다.'],
      dtype='<U896')

In [17]:
train_data_a = np.array(prot_data[:int(0.8*len(prot_data))], dtype = str)
test_data_a = np.array(prot_data[int(0.8*len(prot_data)):], dtype = str)

In [18]:
cal_len(train_data_a, test_data_a, 0.999999)

spl len is :  240
seg len is :  896


In [19]:
train_input_ids_a, train_attention_mask_a = bert_encode(train_data_a, 240)

In [20]:
print(tokenizer.encode('옌'))

[0, 6, 246458, 2]


In [22]:
train_input_ids_a[1]

array([     0, 193757,  27940, 101359,   7214, 134556,   5144,     32,
            6, 246458,    305,   2166,  20451,   7214,   1963,      2,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
      

In [32]:
def make_laber(data, index_num, max_len):
    return_data = []
    
    for conv in data:
        config_num = 0
        return_sigle_sent = []
        for sen in conv:
            if sen == index_num:
                if config_num == 0:
                    config_num = 1
                else:
                    config_num = 0
            elif sen == 1:
                return_sigle_sent.append(2)
            else :
                return_sigle_sent.append(config_num)
        return_sigle_sent = np.array(return_sigle_sent)
        if return_sigle_sent.shape[0] != max_len:
            add_len = max_len - return_sigle_sent.shape[0]
            num_in = return_sigle_sent[-1]
            for i in range(add_len):
                return_sigle_sent = np.append(return_sigle_sent, int(num_in))
            return_sigle_sent = np.reshape(return_sigle_sent, (1, max_len))
        return_data.append(return_sigle_sent)
    
    return return_data        

In [33]:
train_label_ids = make_laber(train_input_ids_a,246458, 240)

In [34]:
train_label_ids_np = np.concatenate(train_label_ids, axis=0)

In [44]:
train_label_ids_np[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [39]:
changed_data = []

for sentence in proto_data:
    sentence = re.sub(r'[\n\r]+', ' ', sentence)
    changed_data.append(sentence)

proto_data = np.array(changed_data)

In [40]:
train_data_b = np.array(proto_data[:int(0.8*len(proto_data))], dtype = str)
test_data_b = np.array(proto_data[int(0.8*len(proto_data)):], dtype = str)

In [41]:
train_a_input_ids_b, train_a_attention_mask_b = bert_encode(train_data_b, 240)

In [42]:
train_a_input_ids_b[1]

array([     0, 193757,  27940, 101359,   7214, 134556,   5144,     32,
          305,   2166,  20451,   7214,   1963,      2,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
      

In [43]:
train_a_attention_mask_b[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [45]:
use_bert_model = create_model(bert_model, 240)
use_bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 240)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 117653760   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 240, 64)      24640       tf_bert_model[0][13]         

In [70]:
tf.keras.backend.clear_session()

In [71]:
history = use_bert_model.fit([train_a_input_ids_b, train_a_attention_mask_b], train_label_ids_np, validation_split=0.2, epochs = 30, batch_size=8)

Epoch 1/30
556/556 [==============================] - 129s 228ms/step - loss: 0.0417 - accuracy: 0.9828 - val_loss: 0.4795 - val_accuracy: 0.8408
Epoch 2/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0372 - accuracy: 0.9847 - val_loss: 0.4603 - val_accuracy: 0.8462
Epoch 3/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0325 - accuracy: 0.9868 - val_loss: 0.5043 - val_accuracy: 0.8464
Epoch 4/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0288 - accuracy: 0.9887 - val_loss: 0.4814 - val_accuracy: 0.8474
Epoch 5/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0302 - accuracy: 0.9881 - val_loss: 0.5906 - val_accuracy: 0.8409
Epoch 6/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0247 - accuracy: 0.9904 - val_loss: 0.6311 - val_accuracy: 0.8535
Epoch 7/30
556/556 [==============================] - 128s 230ms/step - loss: 0.0228 - accuracy: 0.9912 - val_loss: 0.6517 -

In [56]:
test_data_a

array(['저기 신입은 남자친구있어? 옌 아 없는데요 옌 그럼 저기 과장님어때? 옌 네? 저보다 20살 많으시지않나요 옌 뭐어때 돈만많음되지 잘해봐 옌 아 전 괜찮습니다 옌 에이 왜케 뻗대 주말에뭐해? 혼자 외롭잖아 옌 아.제 스타일이 아니셔셔요 옌 아 참 막내가 건방지게도 말하네 옌 죄송합니다.',
       '조과장 이거 했어? 옌 아 아직 못했습니다 다른 일이 밀려서요 옌 조과장 뭐하는 사람이야? 휴 너 오늘 집 갈 생각 말고 밥도 먹지 말고 다해놔라 옌 네 이사님 옌 조과장은 할 줄 아는게 없어? 씨 욕나온다 옌 저도 하는데 일이 계속 터져서요 업무 분배해주시면 안될까요? 옌 분배? 조과장이 다해야지 누굴시켜? 내가하리? 옌 아.네 옌 제대로 해라 쫌 할 줄 아는게 뭔지 옌 네.',
       '이거 누가 올린 보고서야!!! 옌 오늘 까지 올리라고 하셔서 제가 아침에 올려놨는데요. 옌 이게 최선이니? 넌 이렇게 밖에 못해? 옆에 있는 김대리랑 동기면서 차이 너무 나는거 아니니? 옌 .뭐가 잘못 됐을까요? 알려주시면 다시 작성해 오겠습니다. 옌 그걸 니가 알아서 해와야지 내가 일일이 그걸 다 알려줘야 하니? 옌 네. 다시 해오겠습니다. 옌 다들 점심먹으러 갑시다 너는 그거 수정해야 해서 점심 못먹지? 우리끼리 갔다올께 옌 네. 다녀오세요. 옌  옌 다했니? 점심도 안먹으면서 했는데도 아직도니? 옌 아네.곧 가져다 드릴께요.',
       ...,
       '애들아 공연 연습하자. 옌 이번주에 최종 평가 있어. 빨리 하자. 옌 야 김정미 너는 왜 맨날 이 부분에서 틀려? 옌 어. 미안해 내가 더 연습해야 했는데 옌 너 때문에 동작이 하나도 안 맞잖아. 그리고 표정은 또 왜이렇게 어두워? 옌 미안해 다음부턴 웃으면서 할게 옌 춤에 소질이 없으면 그냥 팀에서 나가. 너 케어할 시간 없으니까 옌 미안해 내가 더 노력할게 옌 제발 민폐 좀 끼치지마 대체 춤에 뭐가 자신있다고 여기 들어온건지. 옌 알았어.',
       '나를 왜 자꾸 때려 옌 때리고

In [72]:
test_a_input_ids, test_a_attention_mask = bert_encode(test_data_a, 240)

In [73]:
test_label_ids = make_laber(test_a_input_ids,246458, 240)

In [74]:
test_label_ids_np = np.concatenate(test_label_ids, axis=0)

In [76]:
test_a_input_ids_b, test_a_attention_mask_b = bert_encode(test_data_b, 240)

In [79]:
use_bert_model.evaluate([test_a_input_ids_b, test_a_attention_mask_b], test_label_ids_np)

44/44 [==============================] - 9s 209ms/step - loss: 0.8876 - accuracy: 0.8139


[0.8876399993896484, 0.8138608932495117]